In [13]:
import sys
sys.path.insert(0, '../../src')
import feedparser
import pandas as pd

import urllib

from postagem.Util import extract_domain, download_and_move_image, get_noticia_comercio
from lexical_analyzer_package import midia_lexical
from midia_postagem import midia_post
from Model.News import News
from Database import midia_table

from newsplease import NewsPlease
from bs4 import BeautifulSoup
import requests

import datetime

In [14]:
link = 'https://www1.folha.uol.com.br/ultimas-noticias/'

In [15]:
req = requests.get(link)

In [16]:
noticias = BeautifulSoup(req.text, "html.parser").find_all('div', class_='flex-cell')
noticias = BeautifulSoup(req.text, "html.parser").find_all('div', class_='c-headline__wrapper')

In [ ]:
for noticia in noticias:
    print(noticia.find_all('a', href=True)[0]['href'])
    article = NewsPlease.from_url(noticia.find_all('a', href=True)[0]['href'])
    row = {'titulos': [], 'links': [], 'noticia': [], 'image': [], 'abstract': [], 'date': []}
    if (article is not None):
        row['titulos'].append(article.title)
        row['noticia'].append(article.text)
        row['links'].append(article.url)
        row['abstract'].append(article.text)
        row['date'].append(article.date_publish)
        path_image = article.image_url
        if path_image == '' or path_image == None:
            row['image'].append(0)
        else:
            row['image'].append(download_and_move_image(article.image_url))
        news = News(row['abstract'], row['noticia'], row['date'], row['links'], row['titulos'], row['image'])
        try:
            print(row['titulos'])
            news_in_db = midia_table.check_news(news)
            print('news_in_db: ' + str(news_in_db))
            if (not news_in_db):
                row = pd.DataFrame(row)
                df, categories = midia_lexical.lexical_corpus_and_title(row)
                # DB categories
                if (categories != [set()]):
                    news.set_categories(categories)
                    midia_table.save_news(news)
                    midia_post.post_news(df)
        except:
            print('Empty News')

https://www1.folha.uol.com.br/poder/eleicoes/2018
100% [..............................................................................] 34129 / 34129['Eleições 2018']
news_in_db: False
 -- no categories -- 
https://www1.folha.uol.com.br/poder/eleicoes/2018
100% [..............................................................................] 34129 / 34129['Eleições 2018']
news_in_db: False
 -- no categories -- 
https://www1.folha.uol.com.br/mundo/2018/09/sucessos-descritos-na-onu-por-trump-e-temer-nao-existem-no-mundo-real.shtml
100% [............................................................................] 301332 / 301332['Sucessos descritos na ONU por Trump e Temer não existem no mundo real']
news_in_db: False
 -- no categories -- 
https://www1.folha.uol.com.br/ilustrada/2018/09/modelo-do-nu-a-origem-do-mundo-e-finalmente-revelada.shtml
100% [............................................................................] 140763 / 140763["Modelo do nu 'A Origem do Mundo', de 1866, é 

In [ ]:
noticias[0]